## Импортируем библиотеки

In [1]:
import numpy as np
import os
import pandas as pd

## Определим путь к данным

In [2]:
DATA_PATH = os.path.join(os.getcwd(), 'Python', '4.1', 'names')
print(DATA_PATH)

C:\Users\Белоусовы\Python\4.1\names


## Опишем функцию, которая вернет отфильтрованный список файлов по расширению

In [30]:
def get_files_by_ext(files, ext):
    result = {}
    for file_name in files:
        if file_name.endswith("." + ext):
            year = int(os.path.splitext(os.path.basename(file_name))[0][-4:])
            result[year] = os.path.join(DATA_PATH, file_name)
    return result

## Список текстовых файлов:

In [33]:
file_list = get_files_by_ext(os.listdir(DATA_PATH), "txt")
print(file_list[1880])
print(file_list[1881])
print(file_list[1882])
print(file_list[2017])

C:\Users\Белоусовы\Python\4.1\names\yob1880.txt
C:\Users\Белоусовы\Python\4.1\names\yob1881.txt
C:\Users\Белоусовы\Python\4.1\names\yob1882.txt
C:\Users\Белоусовы\Python\4.1\names\yob2017.txt


## Функция получения сгруппированного списка по входным годам

In [49]:
def get_df_concat(years):
    dfs = []
    for year in years:
        dfs.append(pd.read_csv(file_list[year],
                               names=['Name', 'Gender', 'Count'],
                               engine="python"))
    return pd.concat(dfs, ignore_index=True).groupby('Name').sum()
print(get_df_concat([1900, 1950, 2000]).head(3))

         Count
Name          
Aadam        6
Aadarsh      5
Aadil        9


## ТОП n популярных имен

In [69]:
def get_top_n_names(years, n):
    return get_df_concat(years).sort_values('Count', ascending=False).head(n)
df = get_top_n_names([1900, 1950, 2000], 3)
print(df.index.values.tolist())

['James', 'John', 'Robert']


## Функция соединяющая фреймы

In [107]:
def get_df_merge(years):
    df = None
    for year in years:
        if df is None:
            df = pd.read_csv(file_list[year],
                             names=['Name', 'Gender', 'Count_' + str(year)],
                             engine="python")
        else:
            df2 = pd.read_csv(file_list[year],
                              names=['Name', 'Gender', 'Count_' + str(year)],
                              engine="python")
            df = pd.merge(df, df2, on=['Name', 'Gender'], how='outer')
    for year in years:
        df['Count_' + str(year)] = df['Count_' + str(year)].fillna(0).astype(np.int64)
    return df
print(get_df_merge([1900, 1950, 2000]).head(3))

    Name Gender  Count_1900  Count_1950  Count_2000
0   Mary      F       16706       65482        6192
1  Helen      F        6343        7059         890
2   Anna      F        6114        3817       10588


## Динамика изменения количества имен за указанные года в разрезе полов

In [108]:
def count_dynamics(years):    
    df = get_df_merge(years)
    df = df.groupby('Gender').sum()
    print(df.dtypes)
    return df.T.to_dict('list')
print(count_dynamics([1900, 1950, 2000]))

Count_1900    int64
Count_1950    int64
Count_2000    int64
dtype: object
{'F': [299800, 1713151, 1815110], 'M': [150483, 1790549, 1962969]}
